# Comparación red neuronal regresión

## Importar librerías

In [1]:
# ver directorio de trabajo
import os
print(os.getcwd())

/home/jovyan/work


In [2]:
import os
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_breast_cancer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import numpy as np
import tensorflow as tf
import random as random

2026-02-11 12:59:30.799763: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-11 12:59:30.859772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-11 12:59:32.077435: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
# Comprobar GPU y configurar memoria
print(tf.__version__)

gpus = tf.config.list_physical_devices("GPU")
print("GPUs:", gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU habilitada")
    except RuntimeError as e:
        print("Error al configurar GPU:", e)
else:
    print("No se detectaron GPUs")

2.20.0
GPUs: []
No se detectaron GPUs


2026-02-11 12:59:39.046888: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


## Importar datos

In [ ]:
california_housing = fetch_california_housing(as_frame=True)

df = california_housing.frame

## Análisis explotario de datos

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Ver distribución de las variables
import matplotlib.pyplot as plt

df.hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

In [ ]:
# Escalar los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop(columns=['MedHouseVal']))
y = df['MedHouseVal']

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Función para crear el modelo
def build_model(hp):
    model = Sequential()
    
    num_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    
    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=16, max_value=128, step=16)
        activation = hp.Choice(f'activation_{i}', values=['relu', 'tanh', 'sigmoid'])
        if i == 0:
            model.add(Dense(units=units, activation=activation, input_shape=(X_train.shape[1],)))
        else:
            model.add(Dense(units=units, activation=activation))
            
    model.add(Dense(1))  # Capa de salida para regresión
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [3]:
# Comprobar directorio de trabajo
print("Directorio de trabajo:", os.getcwd())

In [ ]:
# Configurar ruta de salida de modelos en el directorio de trabajo 
from pathlib import Path

ROOT = Path(os.getenv('PYTHONPATH'))

In [2]:
# Configurar primer optimizador de hiperparámetros


turner = kt.BayesianOptimization(
    build_model,
    objective='val_mae',
    max_trials=5,
    num_initial_points=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='california_housing_regression',
    seed=42
)

turner.search(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

best_hps = turner.get_best_hyperparameters(num_trials=1)[0]
print(f"Mejores hiperparámetros: {best_hps.values}")

In [ ]:
# Modelo final con los mejores hiperparámetros
model = turner.hypermodel.build(best_hps)

cp = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[cp, es], verbose=0)

test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Precisión en el conjunto de prueba: {test_accuracy[1]}")